## Preparing the data

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os, glob
from matplotlib import cm
import psychopy
from psychopy.tools.filetools import fromFile

/Users/egor/Library/Enthought/Canopy_32bit/User/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


Loading the data:

In [11]:
%matplotlib inline
plt.style.use('dark_background')
expName = 'mcBv3_maskV'
#condName = 'mc2_tgT-mcBv_peri'
dataDir = os.path.abspath(os.path.join(os.getcwd(), '..', '..', 'data_bv3/', expName))
print dataDir
allSubjDirs = glob.glob(dataDir + '/' + '*' + expName + '*')
print allSubjDirs
stairs = pd.DataFrame()
# matrices for intensities, responses, and means of reversals:
intns, resps, revMeans, revIntns = [],[],[],[]
orderInfo = pd.DataFrame() # subj, session, and stair info for ordering intns & resps

/Users/egor/Dropbox/Projects/mc/data_bv3/mcBv3_maskV
['/Users/egor/Dropbox/Projects/mc/data_bv3/mcBv3_maskV/mc2_tgT-mcBv3_maskV-cent_p0_s1_2017-08-15_1304', '/Users/egor/Dropbox/Projects/mc/data_bv3/mcBv3_maskV/mc2_tgT-mcBv3_maskV-peri_p0_s2_2017-08-15_1447']


In [13]:
for subjDir in allSubjDirs:
    print subjDir
    subjStairsPath = glob.glob(subjDir + '/' + '*' + expName + '*.csv')[0]
    subjStairs = pd.read_csv(subjStairsPath)
    stairs = stairs.append(subjStairs)
    print subjStairs
    for subjStairLabel in subjStairs['stairLabel']:
        #print subjStairLabel #thisSubjStair['label']
        thisStair = fromFile(subjDir + '/' + subjStairLabel + '.psydat')
        intns.append(thisStair.intensities)
        resps.append(thisStair.data)
        theseRevIntns = np.array(thisStair.reversalIntensities) #detectRevs(thisStair)
        theseRevIntns[theseRevIntns<-4] = np.nan
        #print np.round(theseRevIntns,1)
        revIntns.append(theseRevIntns[-8:])
        revMeans.append([np.average(theseRevIntns[-4:]),
                         np.average(theseRevIntns[-6:]),
                         np.average(theseRevIntns[-8:])])
        thisStairStart = thisStair.extraInfo['startContr']
        if thisStairStart < -2:
            thisStairStart = -2
        orderInfo = orderInfo.append(pd.DataFrame({
                    'participant': subjStairs.participant[0],
                    'mcBv': thisStair.extraInfo['mcBv'],
                    'targTpeak': thisStair.extraInfo['targTpeak'],
                    'targXoff2': thisStair.extraInfo['targXoff2'],
                    #'targV': thisStair.extraInfo['targV'],
                    'stairStart': [thisStairStart]
                    }))
                    #'sessionLabel': os.path.basename(subjStairsPath)[:-20],
                    #'stairLabel': [subjStairLabel]
        #print np.round(thisStair.intensities,2)
        #print thisStair.data # responses (correct/incorrect)
        #print thisStair.extraInfo

/Users/egor/Dropbox/Projects/mc/data_bv3/mcBv3_maskV/mc2_tgT-mcBv3_maskV-cent_p0_s1_2017-08-15_1304
    startContr  mcSz  mcSf  mcBv  mcBsf  mcPeriFade  targSz  targSf  targOri1  \
0           -2   256   0.1  0.01   0.05           8      28     0.1         0   
1           -2   256   0.1  0.01   0.05           8      28     0.1         0   
2           -2   256   0.1  0.01   0.05           8      28     0.1         0   
3           -2   256   0.1  0.60   0.05           8      28     0.1         0   
4           -2   256   0.1  0.60   0.05           8      28     0.1         0   
5           -2   256   0.1  0.60   0.05           8      28     0.1         0   
6           -2   256   0.1  1.20   0.05           8      28     0.1         0   
7           -2   256   0.1  1.20   0.05           8      28     0.1         0   
8           -2   256   0.1  1.20   0.05           8      28     0.1         0   
9           -2   256   0.1  2.40   0.05           8      28     0.1         0   
10       

KeyError: 'stairLabel'

In [5]:
stairs['condLabel'] = 'mcBv-' + stairs.mcBv.astype(str) + '_' + \
                      'tgT-' + stairs.targTpeak.astype(str) + '_' + \
                      'tgEcc-' + stairs.targXoff2.astype(str) # + '_' + \
                      #'tgV-' + stairs.targV.astype(str)
stairs.ix[stairs.stairStart<-2,'stairStart'] = -2
print stairs.shape
print stairs.columns
stairs.head(5)

AttributeError: 'DataFrame' object has no attribute 'mcBv'

In [5]:
print np.shape(resps)
print np.shape(intns)
print np.shape(revMeans)
orderInfo = orderInfo.reset_index(drop=True)
print orderInfo.shape
orderInfo.head(5)

(420L,)
(420L,)
(420L, 3L)
(420, 5)


,mcBv,participant,stairStart,targTpeak,targXoff2
0,0.01,0,0.0,0.66,28
1,0.01,0,0.0,0.33,28
2,0.01,0,-2.0,0.66,28
3,0.01,0,-2.0,0.33,28
4,0.01,0,-2.0,1.66,28


In [6]:
print np.shape(revMeans)
meanRevsDF = pd.DataFrame({
    'meanRevs4': np.array(revMeans)[:,0],
    'meanRevs6': np.array(revMeans)[:,1],
    'meanRevs8': np.array(revMeans)[:,2] })
orderedRevMeans = pd.concat([meanRevsDF, orderInfo], axis=1)
#orderedRevMeans.head(5)
stairsWithRevs = pd.merge(orderedRevMeans, stairs, on=['participant','mcBv','targTpeak',
                                                       'targXoff2', 'stairStart'])
stairsWithRevs.head(5)

(420L, 3L)


,meanRevs4,meanRevs6,meanRevs8,mcBv,participant,stairStart,targTpeak,targXoff2,expName,expPara,...,targOri2,targXoff1,targYoff,targV,targTtot,trialT,fixCross,stairLabel,meanRev6,condLabel
0,-1.5175,-1.545000,-1.52125,0.01,0,0.0,0.66,28,mc2_tgT-mcBv2,cent,...,0,-28,0,0,0.66,2,1,st0_mcBv0.01_targTpeak0.66,-1.545000,mcBv-0.01_tgT-0.66_tgEcc-28
1,-1.4775,-1.418333,-1.43875,0.01,0,0.0,0.33,28,mc2_tgT-mcBv2,cent,...,0,-28,0,0,0.66,2,1,st0_mcBv0.01_targTpeak0.33,-1.418333,mcBv-0.01_tgT-0.33_tgEcc-28
2,-1.5950,-1.596667,-1.58500,0.01,0,-2.0,0.66,28,mc2_tgT-mcBv2,cent,...,0,-28,0,0,0.66,2,1,st-2_mcBv0.01_targTpeak0.66,-1.596667,mcBv-0.01_tgT-0.66_tgEcc-28
3,-1.2950,-1.396667,-1.43500,0.01,0,-2.0,0.33,28,mc2_tgT-mcBv2,cent,...,0,-28,0,0,0.66,2,1,st-2_mcBv0.01_targTpeak0.33,-1.396667,mcBv-0.01_tgT-0.33_tgEcc-28
4,-1.6950,-1.696667,-1.73500,0.01,0,-2.0,1.66,28,mc2_tgT-mcBv2,cent,...,0,-28,0,0,0.66,2,1,st-2_mcBv0.01_targTpeak1.66,-1.696667,mcBv-0.01_tgT-1.66_tgEcc-28


In [7]:
ptntMeans = stairsWithRevs.groupby(['participant'])['meanRevs4','meanRevs6','meanRevs8'].mean()
ptntMeans['participant'] = ptntMeans.index.get_level_values('participant')
ptntMeans = ptntMeans.reset_index(drop=True)
ptntMeans.columns = ['ptnt4','ptnt6','ptnt8','participant']
ptntMeans.head()

,ptnt4,ptnt6,ptnt8,participant
0,-1.532562,-1.524208,-1.533469,0
1,-1.281438,-1.284292,-1.292594,1
2,-0.947000,-0.964667,-0.966625,2
3,-1.251125,-1.249083,-1.250875,9
4,-1.432062,-1.418458,-1.405094,13


In [8]:
condRevMeans = stairsWithRevs.groupby(['condLabel','participant'])['meanRevs4','meanRevs6','meanRevs8'].mean()
#print condRevMeans.head(6)
#print condRevMeans.index.get_level_values('participant')
#print condRevMeans.index
#condRevMeans['condLabel'] = condRevMeans.index
condRevMeans['participant'] = condRevMeans.index.get_level_values('participant')
condRevMeans['condLabel'] = condRevMeans.index.get_level_values('condLabel')
condRevMeans = condRevMeans.reset_index(drop=True)
print condRevMeans.shape
condRevMeans.head(6)

(210, 5)


,meanRevs4,meanRevs6,meanRevs8,participant,condLabel
0,-1.30500,-1.303333,-1.290000,0,mcBv-0.01_tgT-0.33_tgEcc-100
1,-1.11000,-1.090000,-1.105000,1,mcBv-0.01_tgT-0.33_tgEcc-100
2,-0.79625,-0.780833,-0.810625,2,mcBv-0.01_tgT-0.33_tgEcc-100
3,-0.98500,-0.990000,-1.011250,9,mcBv-0.01_tgT-0.33_tgEcc-100
4,-1.05500,-1.053333,-1.052500,13,mcBv-0.01_tgT-0.33_tgEcc-100
5,-1.29500,-1.280000,-1.297500,14,mcBv-0.01_tgT-0.33_tgEcc-100


In [9]:
condRevNPtntMeans = pd.merge(condRevMeans, ptntMeans, how='inner', on=['participant'])
condRevNPtntMeans['norm4'] = condRevNPtntMeans.ptnt4 / condRevNPtntMeans.meanRevs4
condRevNPtntMeans['norm6'] = condRevNPtntMeans.ptnt6 / condRevNPtntMeans.meanRevs6
condRevNPtntMeans['norm8'] = condRevNPtntMeans.ptnt8 / condRevNPtntMeans.meanRevs8
condRevNPtntMeans.head()

,meanRevs4,meanRevs6,meanRevs8,participant,condLabel,ptnt4,ptnt6,ptnt8,norm4,norm6,norm8
0,-1.30500,-1.303333,-1.290000,0,mcBv-0.01_tgT-0.33_tgEcc-100,-1.532562,-1.524208,-1.533469,1.174377,1.169469,1.188735
1,-1.38625,-1.407500,-1.436875,0,mcBv-0.01_tgT-0.33_tgEcc-28,-1.532562,-1.524208,-1.533469,1.105546,1.082919,1.067225
2,-1.50000,-1.483333,-1.468750,0,mcBv-0.01_tgT-0.66_tgEcc-100,-1.532562,-1.524208,-1.533469,1.021708,1.027556,1.044064
3,-1.55625,-1.570833,-1.553125,0,mcBv-0.01_tgT-0.66_tgEcc-28,-1.532562,-1.524208,-1.533469,0.984779,0.970318,0.987344
4,-1.53000,-1.520000,-1.515000,0,mcBv-0.01_tgT-1.0_tgEcc-100,-1.532562,-1.524208,-1.533469,1.001675,1.002769,1.012191


In [10]:
stairsClean = stairs.drop(['stairStart','meanRev6','stairLabel'],axis=1,inplace=False)
stairsClean.drop_duplicates(inplace=True)
conds = pd.merge(condRevNPtntMeans, stairsClean, how='inner', on=['condLabel','participant'])
conds['Mask\nVelocity'] = np.round(conds.mcBv * 60/35,0)
print conds.columns
conds.head(7)

Index([u'meanRevs4', u'meanRevs6', u'meanRevs8', u'participant', u'condLabel',
       u'ptnt4', u'ptnt6', u'ptnt8', u'norm4', u'norm6', u'norm8', u'expName',
       u'expPara', u'time', u'dom', u'session', u'nRevs', u'mcSz', u'mcSf',
       u'mcBv', u'mcBsf', u'mcPeriGap', u'mcPeriFade', u'targSz', u'targSf',
       u'targOri1', u'targOri2', u'targXoff1', u'targXoff2', u'targYoff',
       u'targV', u'targTtot', u'targTpeak', u'trialT', u'fixCross',
       u'Mask\nVelocity'],
      dtype='object')


,meanRevs4,meanRevs6,meanRevs8,participant,condLabel,ptnt4,ptnt6,ptnt8,norm4,norm6,...,targOri2,targXoff1,targXoff2,targYoff,targV,targTtot,targTpeak,trialT,fixCross,Mask Velocity
0,-1.30500,-1.303333,-1.290000,0,mcBv-0.01_tgT-0.33_tgEcc-100,-1.532562,-1.524208,-1.533469,1.174377,1.169469,...,0,-100,100,0,0,0.66,0.33,2,1,0.0
1,-1.38625,-1.407500,-1.436875,0,mcBv-0.01_tgT-0.33_tgEcc-28,-1.532562,-1.524208,-1.533469,1.105546,1.082919,...,0,-28,28,0,0,0.66,0.33,2,1,0.0
2,-1.50000,-1.483333,-1.468750,0,mcBv-0.01_tgT-0.66_tgEcc-100,-1.532562,-1.524208,-1.533469,1.021708,1.027556,...,0,-100,100,0,0,0.66,0.66,2,1,0.0
3,-1.55625,-1.570833,-1.553125,0,mcBv-0.01_tgT-0.66_tgEcc-28,-1.532562,-1.524208,-1.533469,0.984779,0.970318,...,0,-28,28,0,0,0.66,0.66,2,1,0.0
4,-1.53000,-1.520000,-1.515000,0,mcBv-0.01_tgT-1.0_tgEcc-100,-1.532562,-1.524208,-1.533469,1.001675,1.002769,...,0,-100,100,0,0,0.66,1.00,2,1,0.0
5,-1.71750,-1.645000,-1.652500,0,mcBv-0.01_tgT-1.0_tgEcc-28,-1.532562,-1.524208,-1.533469,0.892322,0.926570,...,0,-28,28,0,0,0.66,1.00,2,1,0.0
6,-1.48875,-1.475833,-1.481875,0,mcBv-0.01_tgT-1.33_tgEcc-100,-1.532562,-1.524208,-1.533469,1.029429,1.032778,...,0,-100,100,0,0,0.66,1.33,2,1,0.0


In [11]:
mtLabels = ['Static mask\nCenter target','Static mask\nPeripheral target',
            'Dynamic mask\nCenter target','Dynamic mask\nPeripheral target']
conds.ix[(conds.targXoff2<50) & (conds.mcBv<1),'Mask/Target'] = mtLabels[0]
conds.ix[(conds.targXoff2>50) & (conds.mcBv<1),'Mask/Target'] = mtLabels[1]
conds.ix[(conds.targXoff2<50) & (conds.mcBv>1),'Mask/Target'] = mtLabels[2]
conds.ix[(conds.targXoff2>50) & (conds.mcBv>1),'Mask/Target'] = mtLabels[3]

In [12]:
conds.to_csv('conds3_maskV.csv')